In [4]:
import numpy as np
import tensorflow as tf
import collections
import os

start_token = 'G'
end_token = 'E'
batch_size = 64

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'

#### 数据预处理部分

In [6]:
def process_poems(file_name):
    poems = []
    with open(file_name, "r", encoding='utf-8', ) as f:
        for line in f.readlines():
            try:
                title, content = line.strip().split(':')
                content = content.replace(' ', '')
                if '_' in content or '(' in content or '（' in content or '《' in content or '[' in content or \
                                start_token in content or end_token in content:
                    continue
                if len(content) < 5 or len(content) > 80:
                    continue
                content = start_token + content + end_token
                poems.append(content)
            except ValueError as e:
                pass
    # 按诗的字数排序
    poems = sorted(poems, key=lambda line: len(line))
    # 统计每个字出现次数
    all_words = []
    for poem in poems:
        all_words += [word for word in poem]  
    counter = collections.Counter(all_words)  # 统计词和词频。
    count_pairs = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*count_pairs)
    words = words[:len(words)] + (' ',)
    word_int_map = dict(zip(words, range(len(words))))
    poems_vector = [list(map(word_int_map.get, poem)) for poem in poems]
    return poems_vector, word_int_map, words

### rnn_lstm model

In [7]:
def rnn_model(model, input_data, output_data, vocab_size, rnn_size=128, num_layers=2, batch_size=64,
              learning_rate=0.01):
    end_points = {}
    # 构建RNN基本单元RNNcell
    if model == 'rnn':
        cell_fun = tf.contrib.rnn.BasicRNNCell
    elif model == 'gru':
        cell_fun = tf.contrib.rnn.GRUCell
    else:
        cell_fun = tf.contrib.rnn.BasicLSTMCell
    #？？？？？？？？？？？？？？？？？？？？？？
    # 每层128个小单元，一共有两层，输出的Ct 和 Ht 要分开放到两个tuple中
    # 在下面补全代码 
    #################################################
    cell = cell_fun(rnn_size, state_is_tuple=True)
    cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
    #################################################
    # 如果是训练模式，output_data不为None，则初始状态shape为[batch_size * rnn_size]
    # 如果是生成模式，output_data为None，则初始状态shape为[1 * rnn_size]
    if output_data is not None:
        initial_state = cell.zero_state(batch_size, tf.float32)
    else:
        initial_state = cell.zero_state(1, tf.float32)

    # 构建隐层
    for d in ['/gpu:0', '/gpu:1']:
        with tf.device(d):
            embedding = tf.Variable(tf.random_uniform([vocab_size + 1, rnn_size], -1.0, 1.0),name = 'embedding')
            inputs = tf.nn.embedding_lookup(embedding, input_data)
    #？？？？？？？？？？？？？？？？？？？？？？？？？？
    ####################################################    
    outputs, last_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)# 填写里面的内容
    ######################################################
    output = tf.reshape(outputs, [-1, rnn_size])
    
    weights = tf.Variable(tf.truncated_normal([rnn_size, vocab_size + 1]))
    bias = tf.Variable(tf.zeros(shape=[vocab_size + 1]))
    logits = tf.nn.bias_add(tf.matmul(output, weights), bias=bias) # 一层全连接


    if output_data is not None: # 训练模式
        labels = tf.one_hot(tf.reshape(output_data, [-1]), depth=vocab_size + 1)
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
        total_loss = tf.reduce_mean(loss)
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)  # 优化器用的 adam
        end_points['initial_state'] = initial_state
        end_points['output'] = output
        end_points['train_op'] = train_op
        end_points['total_loss'] = total_loss
        end_points['loss'] = loss
        end_points['last_state'] = last_state
    else: # 生成模式
        prediction = tf.nn.softmax(logits)
        end_points['initial_state'] = initial_state
        end_points['last_state'] = last_state
        end_points['prediction'] = prediction
    return end_points

### 训练模型部分

In [8]:
def run_training(save_file):
    # 处理数据集
    poems_vector, word_to_int, vocabularies = process_poems('./poems.txt')
    # 生成batch
    batches_inputs, batches_outputs = generate_batch(64, poems_vector, word_to_int)

    input_data = tf.placeholder(tf.int32, [batch_size, None])
    output_targets = tf.placeholder(tf.int32, [batch_size, None])
    # 构建模型
    end_points = rnn_model(model='lstm', input_data=input_data, output_data=output_targets, vocab_size=len(
        vocabularies), rnn_size=128, num_layers=2, batch_size=64, learning_rate=0.01)

    saver = tf.train.Saver()
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    with tf.Session() as sess:
        sess.run(init_op)
        for epoch in range(500):
            n = 0
            n_chunk = len(poems_vector) // batch_size
            for batch in range(n_chunk):
                loss, _, _ = sess.run([
                    end_points['total_loss'],
                    end_points['last_state'],
                    end_points['train_op']
                ], feed_dict={input_data: batches_inputs[n], output_targets: batches_outputs[n]})
                n += 1
                print('[INFO] Epoch: %d , batch: %d , training loss: %.6f' % (epoch, batch, loss))
            if loss < 3.0:
                break
#         saver.save(sess, './poem_generator')
        saver.save(sess, save_file)        

### 生成 诗歌部分

In [9]:
def gen_poem(begin_word,save_file):
    batch_size = 1
    poems_vector, word_int_map, vocabularies = process_poems('./poems.txt')

    input_data = tf.placeholder(tf.int32, [batch_size, None])

    end_points = rnn_model(model='lstm', input_data=input_data, output_data=None, vocab_size=len(
        vocabularies), rnn_size=128, num_layers=2, batch_size=64, learning_rate=0.01)

    
        
    saver = tf.train.Saver(tf.global_variables())
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

    with tf.Session() as sess:
        sess.run(init_op)
        saver.restore(sess, save_file)# 恢复之前训练好的模型 
        poem = ''
        #???????????????????????????????????????
        # 下面部分代码主要功能是根据指定的开始字符来生成诗歌
        #########################################
        x = np.array([list(map(word_int_map.get, start_token))])
        [predict, last_state] = sess.run([end_points['prediction'], end_points['last_state']],
                                         feed_dict={input_data: x})
        word = begin_word
        while word != end_token:
            poem += word
            x = np.zeros((1, 1))
            x[0, 0] = word_int_map[word]
            [predict, last_state] = sess.run([end_points['prediction'], end_points['last_state']],
                                             feed_dict={input_data: x, end_points['initial_state']: last_state})
            word = to_word(predict, vocabularies)
        
        #########################################
        return poem

### 其他的一些处理函数

In [ ]:
def generate_batch(batch_size, poems_vec, word_to_int):
    # 每次取64首诗进行训练
    n_chunk = len(poems_vec) // batch_size
    x_batches = []
    y_batches = []
    for i in range(n_chunk):
        start_index = i * batch_size
        end_index = start_index + batch_size

        batches = poems_vec[start_index:end_index]
        # 找到这个batch的所有poem中最长的poem的长度
        length = max(map(len, batches))
        # 填充一个这么大小的空batch，空的地方放空格对应的index标号
        x_data = np.full((batch_size, length), word_to_int[' '], np.int32)
        for row in range(batch_size):
            x_data[row, :len(batches[row])] = batches[row]
        y_data = np.copy(x_data)
        y_data[:, :-1] = x_data[:, 1:]
        """
        x_data             y_data
        [6,2,4,6,9]       [2,4,6,9,9]
        [1,4,2,8,5]       [4,2,8,5,5]
        """
        x_batches.append(x_data)
        y_batches.append(y_data)
    return x_batches, y_batches

def to_word(predict, vocabs):# 预测的结果转化成汉字
    sample = np.argmax(predict)
    if sample > len(vocabs):
        sample = len(vocabs) - 1
    return vocabs[sample]
def pretty_print_poem(poem):#  令打印的结果更工整
    poem_sentences = poem.split('。')
    for s in poem_sentences:
        if s != '' and len(s) > 10:
            print(s + '。')

### 主函数

In [ ]:
tf.reset_default_graph()
print('[INFO] train tang poem...')
run_training('./poem_generator_500_epoch') # 训练模型
#训练模型时间比较长，训练模型完成后每次生成诗歌的时，不需要再次训练 ，可以注销上面的 run_training()。生成部分执行速度很快

[INFO] train tang poem...
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

[INFO] Epoch: 0 , batch: 0 , training loss: 8.797020
[INFO] Epoch: 0 , batch: 1 , training loss: 7.284378
[INFO] Epoch: 0 , batch: 2 , training loss: 10.441524
[INFO] Epoch: 0 , batch: 3 , training loss: 6.526995
[INFO] Epoch: 0 , batch: 4 , training loss: 7.754235
[INFO] Epoch: 0 , batch: 5 , training loss: 6.691163
[INFO] Epoch: 0 , batch: 6 , training loss: 7.491658
[INFO] Epoch: 0 , batch: 7 , training loss: 7.384674
[INFO] Epoch: 0 , batch: 8 , training loss: 7.370260
[INFO] Epoch: 0 , batch: 9 , training loss: 7.287445
[INFO] Epoch: 0 , ba

[INFO] Epoch: 0 , batch: 135 , training loss: 5.556635
[INFO] Epoch: 0 , batch: 136 , training loss: 5.706733
[INFO] Epoch: 0 , batch: 137 , training loss: 5.623903
[INFO] Epoch: 0 , batch: 138 , training loss: 5.784577
[INFO] Epoch: 0 , batch: 139 , training loss: 6.247742
[INFO] Epoch: 0 , batch: 140 , training loss: 6.260918
[INFO] Epoch: 0 , batch: 141 , training loss: 6.037811
[INFO] Epoch: 0 , batch: 142 , training loss: 5.755075
[INFO] Epoch: 0 , batch: 143 , training loss: 5.724460
[INFO] Epoch: 0 , batch: 144 , training loss: 5.766144
[INFO] Epoch: 0 , batch: 145 , training loss: 5.838947
[INFO] Epoch: 0 , batch: 146 , training loss: 5.862205
[INFO] Epoch: 0 , batch: 147 , training loss: 5.574625
[INFO] Epoch: 0 , batch: 148 , training loss: 5.574781
[INFO] Epoch: 0 , batch: 149 , training loss: 5.642587
[INFO] Epoch: 0 , batch: 150 , training loss: 5.891544
[INFO] Epoch: 0 , batch: 151 , training loss: 5.503241
[INFO] Epoch: 0 , batch: 152 , training loss: 5.512493
[INFO] Epo

[INFO] Epoch: 0 , batch: 284 , training loss: 5.326487
[INFO] Epoch: 0 , batch: 285 , training loss: 5.403841
[INFO] Epoch: 0 , batch: 286 , training loss: 5.340783
[INFO] Epoch: 0 , batch: 287 , training loss: 5.139217
[INFO] Epoch: 0 , batch: 288 , training loss: 5.309906
[INFO] Epoch: 0 , batch: 289 , training loss: 5.307545
[INFO] Epoch: 0 , batch: 290 , training loss: 5.211391
[INFO] Epoch: 0 , batch: 291 , training loss: 5.172017
[INFO] Epoch: 0 , batch: 292 , training loss: 5.217531
[INFO] Epoch: 0 , batch: 293 , training loss: 5.219745
[INFO] Epoch: 0 , batch: 294 , training loss: 5.708445
[INFO] Epoch: 0 , batch: 295 , training loss: 5.514990
[INFO] Epoch: 0 , batch: 296 , training loss: 5.409861
[INFO] Epoch: 0 , batch: 297 , training loss: 5.319449
[INFO] Epoch: 0 , batch: 298 , training loss: 5.254310
[INFO] Epoch: 0 , batch: 299 , training loss: 5.144796
[INFO] Epoch: 0 , batch: 300 , training loss: 5.164272
[INFO] Epoch: 0 , batch: 301 , training loss: 5.200540
[INFO] Epo

[INFO] Epoch: 0 , batch: 434 , training loss: 5.308324
[INFO] Epoch: 0 , batch: 435 , training loss: 5.556766
[INFO] Epoch: 0 , batch: 436 , training loss: 5.649966
[INFO] Epoch: 0 , batch: 437 , training loss: 5.509487
[INFO] Epoch: 0 , batch: 438 , training loss: 5.226362
[INFO] Epoch: 0 , batch: 439 , training loss: 5.436584
[INFO] Epoch: 0 , batch: 440 , training loss: 5.532819
[INFO] Epoch: 0 , batch: 441 , training loss: 5.536446
[INFO] Epoch: 0 , batch: 442 , training loss: 5.455103
[INFO] Epoch: 0 , batch: 443 , training loss: 5.587167
[INFO] Epoch: 0 , batch: 444 , training loss: 5.359671
[INFO] Epoch: 0 , batch: 445 , training loss: 5.197024
[INFO] Epoch: 0 , batch: 446 , training loss: 5.040317
[INFO] Epoch: 0 , batch: 447 , training loss: 5.320418
[INFO] Epoch: 0 , batch: 448 , training loss: 5.463908
[INFO] Epoch: 0 , batch: 449 , training loss: 5.751895
[INFO] Epoch: 0 , batch: 450 , training loss: 5.734023
[INFO] Epoch: 0 , batch: 451 , training loss: 5.621028
[INFO] Epo

[INFO] Epoch: 1 , batch: 41 , training loss: 4.976492
[INFO] Epoch: 1 , batch: 42 , training loss: 5.557801
[INFO] Epoch: 1 , batch: 43 , training loss: 5.455974
[INFO] Epoch: 1 , batch: 44 , training loss: 5.417092
[INFO] Epoch: 1 , batch: 45 , training loss: 5.852530
[INFO] Epoch: 1 , batch: 46 , training loss: 6.655379
[INFO] Epoch: 1 , batch: 47 , training loss: 5.337350
[INFO] Epoch: 1 , batch: 48 , training loss: 5.490172
[INFO] Epoch: 1 , batch: 49 , training loss: 5.376513
[INFO] Epoch: 1 , batch: 50 , training loss: 5.337482
[INFO] Epoch: 1 , batch: 51 , training loss: 5.164062
[INFO] Epoch: 1 , batch: 52 , training loss: 4.954906
[INFO] Epoch: 1 , batch: 53 , training loss: 5.132703
[INFO] Epoch: 1 , batch: 54 , training loss: 5.074963
[INFO] Epoch: 1 , batch: 55 , training loss: 5.135817
[INFO] Epoch: 1 , batch: 56 , training loss: 4.960893
[INFO] Epoch: 1 , batch: 57 , training loss: 4.794795
[INFO] Epoch: 1 , batch: 58 , training loss: 4.819805
[INFO] Epoch: 1 , batch: 59 

...

[INFO] Epoch: 70 , batch: 146 , training loss: 4.000794
[INFO] Epoch: 70 , batch: 147 , training loss: 3.658469
[INFO] Epoch: 70 , batch: 148 , training loss: 3.642023
[INFO] Epoch: 70 , batch: 149 , training loss: 3.713154
[INFO] Epoch: 70 , batch: 150 , training loss: 3.957096
[INFO] Epoch: 70 , batch: 151 , training loss: 3.824078
[INFO] Epoch: 70 , batch: 152 , training loss: 3.841888
[INFO] Epoch: 70 , batch: 153 , training loss: 3.874768
[INFO] Epoch: 70 , batch: 154 , training loss: 3.975493
[INFO] Epoch: 70 , batch: 155 , training loss: 4.190573
[INFO] Epoch: 70 , batch: 156 , training loss: 3.901084
[INFO] Epoch: 70 , batch: 157 , training loss: 3.851059
[INFO] Epoch: 70 , batch: 158 , training loss: 3.976101
[INFO] Epoch: 70 , batch: 159 , training loss: 3.901814
[INFO] Epoch: 70 , batch: 160 , training loss: 4.102332
[INFO] Epoch: 70 , batch: 161 , training loss: 4.153212
[INFO] Epoch: 70 , batch: 162 , training loss: 4.143347
[INFO] Epoch: 70 , batch: 163 , training loss: 4

[INFO] Epoch: 70 , batch: 295 , training loss: 4.171139
[INFO] Epoch: 70 , batch: 296 , training loss: 4.099365
[INFO] Epoch: 70 , batch: 297 , training loss: 4.080554
[INFO] Epoch: 70 , batch: 298 , training loss: 3.926401
[INFO] Epoch: 70 , batch: 299 , training loss: 3.944996
[INFO] Epoch: 70 , batch: 300 , training loss: 3.942158
[INFO] Epoch: 70 , batch: 301 , training loss: 3.858130
[INFO] Epoch: 70 , batch: 302 , training loss: 4.026115
[INFO] Epoch: 70 , batch: 303 , training loss: 4.038955
[INFO] Epoch: 70 , batch: 304 , training loss: 4.172995
[INFO] Epoch: 70 , batch: 305 , training loss: 4.021776
[INFO] Epoch: 70 , batch: 306 , training loss: 4.141798
[INFO] Epoch: 70 , batch: 307 , training loss: 4.151932
[INFO] Epoch: 70 , batch: 308 , training loss: 3.967978
[INFO] Epoch: 70 , batch: 309 , training loss: 3.947356
[INFO] Epoch: 70 , batch: 310 , training loss: 3.894505
[INFO] Epoch: 70 , batch: 311 , training loss: 3.883110
[INFO] Epoch: 70 , batch: 312 , training loss: 3

[INFO] Epoch: 70 , batch: 442 , training loss: 4.284900
[INFO] Epoch: 70 , batch: 443 , training loss: 4.434328
[INFO] Epoch: 70 , batch: 444 , training loss: 4.078905
[INFO] Epoch: 70 , batch: 445 , training loss: 3.961398
[INFO] Epoch: 70 , batch: 446 , training loss: 3.899685
[INFO] Epoch: 70 , batch: 447 , training loss: 4.103964
[INFO] Epoch: 70 , batch: 448 , training loss: 4.217112
[INFO] Epoch: 70 , batch: 449 , training loss: 4.599840
[INFO] Epoch: 70 , batch: 450 , training loss: 4.678648
[INFO] Epoch: 70 , batch: 451 , training loss: 4.580640
[INFO] Epoch: 70 , batch: 452 , training loss: 4.400016
[INFO] Epoch: 70 , batch: 453 , training loss: 4.127885
[INFO] Epoch: 70 , batch: 454 , training loss: 4.313208
[INFO] Epoch: 70 , batch: 455 , training loss: 4.348167
[INFO] Epoch: 70 , batch: 456 , training loss: 4.349704
[INFO] Epoch: 70 , batch: 457 , training loss: 4.442665
[INFO] Epoch: 70 , batch: 458 , training loss: 4.183940
[INFO] Epoch: 70 , batch: 459 , training loss: 4

[INFO] Epoch: 71 , batch: 47 , training loss: 3.648779
[INFO] Epoch: 71 , batch: 48 , training loss: 3.710926
[INFO] Epoch: 71 , batch: 49 , training loss: 3.909215
[INFO] Epoch: 71 , batch: 50 , training loss: 3.648517
[INFO] Epoch: 71 , batch: 51 , training loss: 3.837903
[INFO] Epoch: 71 , batch: 52 , training loss: 3.640787
[INFO] Epoch: 71 , batch: 53 , training loss: 3.807535
[INFO] Epoch: 71 , batch: 54 , training loss: 3.786585
[INFO] Epoch: 71 , batch: 55 , training loss: 3.846004
[INFO] Epoch: 71 , batch: 56 , training loss: 3.703142
[INFO] Epoch: 71 , batch: 57 , training loss: 3.610433
[INFO] Epoch: 71 , batch: 58 , training loss: 3.686702
[INFO] Epoch: 71 , batch: 59 , training loss: 3.730376
[INFO] Epoch: 71 , batch: 60 , training loss: 3.641569
[INFO] Epoch: 71 , batch: 61 , training loss: 3.777334
[INFO] Epoch: 71 , batch: 62 , training loss: 3.644519
[INFO] Epoch: 71 , batch: 63 , training loss: 3.825909
[INFO] Epoch: 71 , batch: 64 , training loss: 4.021470
[INFO] Epo

[INFO] Epoch: 71 , batch: 198 , training loss: 4.054200
[INFO] Epoch: 71 , batch: 199 , training loss: 4.199848
[INFO] Epoch: 71 , batch: 200 , training loss: 4.067529
[INFO] Epoch: 71 , batch: 201 , training loss: 3.976644
[INFO] Epoch: 71 , batch: 202 , training loss: 3.971895
[INFO] Epoch: 71 , batch: 203 , training loss: 4.120359
[INFO] Epoch: 71 , batch: 204 , training loss: 4.168664
[INFO] Epoch: 71 , batch: 205 , training loss: 3.800377
[INFO] Epoch: 71 , batch: 206 , training loss: 3.732292
[INFO] Epoch: 71 , batch: 207 , training loss: 3.720130
[INFO] Epoch: 71 , batch: 208 , training loss: 4.044847
[INFO] Epoch: 71 , batch: 209 , training loss: 4.038990
[INFO] Epoch: 71 , batch: 210 , training loss: 4.022948
[INFO] Epoch: 71 , batch: 211 , training loss: 4.021272
[INFO] Epoch: 71 , batch: 212 , training loss: 4.123607
[INFO] Epoch: 71 , batch: 213 , training loss: 4.075883
[INFO] Epoch: 71 , batch: 214 , training loss: 4.138929
[INFO] Epoch: 71 , batch: 215 , training loss: 4

In [ ]:
tf.reset_default_graph()
print('[INFO] write tang poem...')
poem2 = gen_poem('洋','./poem_generator')# 生成诗歌
print("#" * 25)
pretty_print_poem(poem2)
print('#' * 25)